In [1]:
# The following piece of code gives the opportunity to show multiple outputs
# in one cell:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# Colorful outputs
class bcolors:
    RED       = '\033[91m'
    OKBLUE    = '\033[94m'
    BOLD      = '\033[1m'
    UNDERLINE = '\033[4m'
    ENDC      = '\033[0m'

- The objective of this exercise is to understand the notions of underfitting and overfitting. Using cross-validation on simulated data.

##### Synthetic data
- I have generated 100 points equally distanced from -20 to 20 and save them in a `numpy` array `x`.
- I have created a new numpy array `y` defined as $y_i=\mathrm{sin}(x_i) + 0.05 x_i^3 + \varepsilon_i$ where $\varepsilon_i \sim \mathcal{N}(0,100^2)$, for $i=1,...,100$.
- I have plotted the scatter plot of `x` and `y`.
##### Underfitting vs. Overfitting
-I have fitted a linear regression model and called it `model1` : $y=\beta_0 + \beta_1 x$ and added the fitted line over the scatter plot and computed the mean squared error of `model1`( `sklearn.metrics.mean_squared_error`).
- A polynomial regression model is fitted with degree 5 and called `model2` : $y=\beta_0 + \beta_1 x + \beta_2 x^2 + ... + \beta_5 x^5$ (`sklearn.preprocessing.PolynomialFeatures` in order to create a **new** input array that includes $x^0$, $x^1$, $x^2$, $x^3$ ...).
- The fitted curve is added over the scatter plot and compared `model2` and `model1`.
- I have compute the mean squared error of `model2`.
- I have fitted a polynomial regression model with degree 20 and called it `model3` : $y=\beta_0 + \beta_1 x + \beta_2 x^2 + ... + \beta_{20} x^{20}$.
- I have added the fitted curve over the scatter plot and compared the models.

##### Cross-Validation
- Using $10$-fold cross-validation, I have computed the **averaged validation** mean squared errors for all possible polynomial models by varying the degree of the polynomial model from $1$ to $20$.

##### LASSO
- Using $10$-fold cross-validation and the LASSO regularization, I have fitted the polynomial model with degree 20 and call it `model4`.

In [2]:
##Synthetic data
#Generate 100 points
import numpy as np
import math
p1=-20
p2=20
parts=100
x=np.linspace(p1, p2, parts)
#Create y
y=[]
eps=np.random.normal(0,100, 100)
for i in range(0,100):
    y_i=math.sin(x[i])+(0.05*x[i]**3)+eps[i]
    y.append(y_i)
#Plot Scatter of x,y
import matplotlib.pyplot as plt
plt.figure(1)
plt.scatter(x,y)
#We can fit a linear model over data, however, to be more accurate it is better to use a nonlinear model

##Underfitting vs. Overfitting
import pandas as pd
x = pd.DataFrame(data=x)
y = pd.DataFrame(data=y)
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
# fit the model and feed the data
lr.fit(X = x, y = y)
# regression parameters intercept is beta0 and coef_ is beta1
print(lr.intercept_, lr.coef_)
plt.figure(2)
plt.plot(x, y, 'or', mfc='none');
# add a regression line
plt.plot(x, lr.intercept_+lr.coef_*x, "-b");
plt.xlabel('x');
plt.ylabel('y');
#Mean Squard Error
model1=lr.intercept_+lr.coef_*x
import sklearn
print(sklearn.metrics.mean_squared_error(y,model1))
##Fit a degree five model
x=np.linspace(p1, p2, parts)
#Create y
y=[]
eps=np.random.normal(0,100, 100)
for i in range(0,100):
    y_i=math.sin(x[i])+(0.05*x[i]**3)+eps[i]
    y.append(y_i)
import numpy.polynomial.polynomial as poly
np.polyfit(x,y,5)
coefs = poly.polyfit(x, y, 5)
model2 = poly.polyval(x, coefs)
plt.plot(x, model2)
print(sklearn.metrics.mean_squared_error(y,model2))
##Fit a degree 20 model
x=np.linspace(p1, p2, parts)
#Create y
y=[]
eps=np.random.normal(0,100, 100)
for i in range(0,100):
    y_i=math.sin(x[i])+(0.05*x[i]**3)+eps[i]
    y.append(y_i)
import numpy.polynomial.polynomial as poly
np.polyfit(x,y,20)
coefs = poly.polyfit(x, y, 20)
model3 = poly.polyval(x, coefs)
plt.plot(x, model3)
print(sklearn.metrics.mean_squared_error(y,model2))
##compute the error for all possible degrees
x=np.linspace(p1, p2, parts)
#Create y
y=[]
eps=np.random.normal(0,100, 100)
for i in range(0,100):
    y_i=math.sin(x[i])+(0.05*x[i]**3)+eps[i]
    y.append(y_i)
import numpy.polynomial.polynomial as poly
plt.figure(3)
error=[]
for j in range(1,21):
    np.polyfit(x,y,j)
    coefs = poly.polyfit(x, y, j)
    model = poly.polyval(x, coefs)
    #plt.plot(x, model)
    error.append(sklearn.metrics.mean_squared_error(y,model))
    plt.scatter(j, sklearn.metrics.mean_squared_error(y,model))
plt.xlabel('Degree of Polynomial');
plt.ylabel('Error');
#Lasso and fitmodel
from sklearn import linear_model
kk=np.linspace(0.04, 0.09, 10)
plt.figure(4)
for k in kk:
 reg = linear_model.Lasso(alpha=k,normalize=True)
 x = pd.DataFrame(data=x)
 reg.fit(x,model3)
 model4=reg.predict(x)
 #plt.plot(x,model3)
 #plt.plot(x,model4)
 #print("R Squared:", reg.score(x, model3))
 from sklearn.metrics import mean_squared_error
# print("MSE:", mean_squared_error(model3, model4))
 plt.scatter(k,reg.score(x, model3))
#I select alpha=0.04 to have the highest R-square

## 10-fold CV, with shuffle
from sklearn import model_selection
import numpy as np
# 10-fold CV, with shuffle
y=np.reshape(y,(100,1))
y = pd.DataFrame(data=y)
print("y")
print(y)
n = len(x)
kf_10 = model_selection.KFold(n_splits=10, shuffle=True)

regr = LinearRegression()
r2 = []

# Calculate R2 with only the intercept 
#(no principal components in regression)
score = model_selection.cross_val_score(regr, np.ones((n,1)), y.iloc[:,0], cv=kf_10, scoring='r2').mean()    
r2.append(score)

# Calculate R2 using CV for the 5 principle components, 
#adding one component at the time.
from sklearn.preprocessing import PolynomialFeatures

for i in np.arange(1, 6):
    score = model_selection.cross_val_score(regr, PolynomialFeatures(i).fit_transform(x), y, cv=kf_10, scoring='r2').mean()
    r2.append(score)
    
# Plot results
plt.figure(5)
plt.plot(r2, '-v')
plt.xlabel('Number of principal components in regression')
plt.ylabel('R Squared')

<Figure size 640x480 with 0 Axes>

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

[3.66190349] [[11.95414299]]


<Figure size 640x480 with 0 Axes>

Text(0.5,0,'x')

...

Lasso(alpha=0.09, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

y
             0
0  -214.296998
1  -129.437582
2  -582.346913
3  -369.926407
4  -316.293375
5  -267.736544
6  -189.542802
7  -235.511951
8  -235.993261
9  -137.251949
10 -339.674889
11 -192.961547
12 -117.831456
13 -242.155744
14  -52.365222
15 -114.275928
16    8.619030
17  -52.107157
18   39.757536
19  -70.812421
20  -25.020131
21  -79.047750
22 -419.863466
23  -55.506041
24  -19.735754
25 -261.190576
26 -121.863182
27 -240.049682
28  -26.234767
29 -109.623098
..         ...
70   66.292207
71   80.494865
72   68.635924
73   17.307834
74  169.157489
75  400.582003
76   36.269940
77  168.023248
78 -167.036652
79  -28.222998
80   97.165826
81  133.570698
82  371.169815
83  -25.622123
84  180.964236
85   81.205368
86  155.211743
87  128.732937
88  372.869401
89   62.783739
90  490.698355
91  100.696307
92  215.587252
93  311.321070
94  338.175580
95  223.028454
96  272.083914
97  419.514634
98  384.264205
99  438.284256

[100 rows x 1 columns]


<Figure size 640x480 with 0 Axes>

Text(0.5,0,'Number of principal components in regression')

Text(0,0.5,'R Squared')